In [ ]:
import cv2
import os
import numpy as np
import mediapipe as mp
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import time
from torch.utils.data import DataLoader,TensorDataset, SubsetRandomSampler, ConcatDataset
from torchvision.transforms import ToTensor
from torchvision import datasets
import torch
from torch import nn

labels = {'A':0, 'B':1, 'C':2,'D':3, 'del':4,'E':5, 'F':6, 'G':7, 'H':8, 'I':9, 'J':10, 'K':11, 'L':12, 'M':13, 'N':14,'nothing':15, 'O':16, 'P':17, 'Q':18, 'R':19, 'S':20, 'space':21,'T':22, 'U':23, 'V':24, 'W':25, 'X':26, 'Y':27, 'Z':28}
#Hand detection module
mpHands = mp.solutions.hands
mpDraw = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_model = mpHands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.5)


def get_box(image,landmarks):
    image_width, image_height = image.shape[1], image.shape[0]
    landmark_array = np.empty((0, 2), int)
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        landmark_point = [np.array((landmark_x, landmark_y))]
        landmark_array = np.append(landmark_array, landmark_point, axis=0)
    x, y, w, h = cv2.boundingRect(landmark_array)
    if x-10>0 and y-10>0 and x+w+10<image.shape[1] and y+h+10<image.shape[0]:
        return [x-10, y-10, x + w + 10, y + h +10]
    else:
        if x-10<=0:
            x-=x-10
        if y-10<=0:
            y-=y-10
        if x+w+10>=image.shape[1]:
            w-= x+w+10-image.shape[1]
        if y+h+10>=image.shape[0]:
            h-=y+h+10-image.shape[0]
        return [x-10, y-10, x + w + 10, y + h +10]
def process_data(hand_landmarks):
    new_img = np.ones((250,250,3),dtype=np.uint8)
    mpDraw.draw_landmarks(
        new_img,
        hand_landmarks,
        mpHands.HAND_CONNECTIONS,
        mp_drawing_styles.get_default_hand_landmarks_style(),
        mp_drawing_styles.get_default_hand_connections_style())
    rect = get_box(new_img,hand_landmarks)
    hand = new_img[rect[1]:rect[3],rect[0]:rect[2]]
    hand = np.array(hand)
    hand = cv2.resize(hand,(75,75))
    return hand
    

def filter_data(img,filename,letter):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    results = mp_model.process(img)
    if results.multi_hand_landmarks is not None:
        for hand_landmarks in results.multi_hand_landmarks:
            new_img = process_data(hand_landmarks)
            directory = os.path.join(os.getcwd(),'new_dataset',letter)
            if not os.path.isdir(directory):
                os.mkdir(directory)
            os.chdir(directory)
            cv2.imwrite(filename, new_img)

def get_data(folder):
    base_dir = os.getcwd()
    listdir = os.listdir(folder)
    if not os.path.isdir('new_dataset'):
        os.mkdir('new_dataset')
    for i in range(len(listdir)):
        os.chdir(base_dir)
        sublistdir = os.listdir(os.path.join(folder,listdir[i]))
        for j in range (len(sublistdir)) :
            os.chdir(base_dir)
            path = os.path.join(folder,listdir[i],sublistdir[j])
            img = cv2.imread(path,0)
            filter_data(img,sublistdir[j],listdir[i])
    os.chdir(base_dir)
    
get_data('dataset/asl_alphabet_train')